Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

## Results
|Instance|Universe size|Number of sets|Density|Number of steps to reach solution|Cost|
|:--:|:--:  |:--: |:--:|:--: |:--:      |
|1   |100   |10   |0.2 |29   |283.75    |
|2   |1000  |100  |0.2 |203  |7100.83   |
|3   |10000 |1000 |0.2 |4221 |128865.88 |
|4   |100000|10000|0.1 |55700|1882996.59|
|5   |100000|10000|0.2 |56901|2154916.13|
|6   |100000|10000|0.3 |71591|2187407.29|

## Definitive solution
Speedup is required.
They are used:
- single mutation tweak
- data structure to keep track of number of sets covering each element
- heuristic to limit research space
- random start

### Collaborations
Following parts:
- tweak function
- fitness plot

done in collaboration with Vincenzo Avantaggiato

### Other strategies
Other strategies have been tried, in general they are good, but too slow.

They are shown in *strategies.ipynb*

### Imports

In [ ]:
from random import random, seed
from itertools import product, accumulate
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from icecream import ic

### Data

In [80]:
universe_sizes = [100, 1000, 10_000, 100_000, 100_000, 100_000]
num_sets_sizes = [10, 100, 1000, 10_000, 10_000, 10_000]
densities = [.2, .2, .2, .1, .2, .3]

INIT_DENSITY = 0.5

#### Generator function

In [81]:
def generate_data(universe_size, num_sets, density):
    SETS = np.random.random((num_sets, universe_size)) < density
    for s in range(universe_size):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(num_sets), s] = True
    COSTS = np.pow(SETS.sum(axis=1), 1.1)
    
    return SETS, COSTS

### Helper functions

In [82]:
def init_sol(num_sets: int) -> np.ndarray:
    solution = np.random.random(num_sets) < INIT_DENSITY
    
    return solution

def valid(sets, solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    phenotype = np.logical_or.reduce(sets[solution])     # at least each element covered by a set
    return np.all(phenotype)                             # all elements are covered

def coverage(sets, solution):
    """Returns the number of covered elements in the universe"""
    phenotype = np.logical_or.reduce(sets[solution])    # at least each element covered by a set
    return np.sum(phenotype)                            # number of covered elements    

def cost(costs, solution):
    """Returns the cost of a solution (to be minimized)"""
    return costs[solution].sum()

def fitness(covering: np.ndarray, costs: np.ndarray, solution: np.ndarray):
    """Returns the fitness of the given solution"""
    return (np.sum(covering > 0), -cost(costs, solution))

# Based on Vincenzo Avantaggiato's version
def tweak(solution: np.ndarray, covering: np.ndarray, sets: np.ndarray) -> np.ndarray:
    """Uses a single mutation method"""
    
    new_sol = solution.copy()
    
    index = np.random.randint(0, solution.shape[0])    
    new_sol[index] = not new_sol[index]
    
    modification = 2*new_sol[index]-1
    
    # Rollbacks the modification (if it is a removal): not allowed to remove a set from the solution
    # if its removal causes an element to become uncovered
    if modification == -1 and np.sum(covering[sets[index]] <= 1):
        return solution
        
    # Store the number of sets covering each element in the universe
    covering += modification * sets[index]
    
    """
    mask = np.random.random(solution.shape[0])
    
    new_sol = np.logical_xor(solution, mask)
    
    # Get value of previous solutions whose value changed from 1 to 0
    removed = np.logical_and(solution, mask)
    
    # Get value of previous solutions whose value changed from 0 to 1
    added = np.logical_and(np.logical_not(solution), mask)
    
    removed_mask = np.sum(np.logical_and(sets, removed.reshape(removed.shape[0], 1)), axis=0)
    
    if np.any(np.sum(covering[removed_mask] <= 1, axis=0)):
        return solution
    
    covering += np.sum(np.logical_and(sets, added.reshape(added.shape[0], 1)), axis=0)
    covering -= removed_mask
    """
    return new_sol

In [83]:
def solve_set_cover(sets: np.ndarray, costs: np.ndarray, num_steps: int = 10_000, buf_size: int = 5, init_strength: float = 0.5):
    
    num_sets = sets.shape[0]
    universe_size = sets.shape[1]
    
    solution = init_sol(num_sets)
    best_cov = np.sum(sets[solution], axis=0)
    sol_fitness = fitness(best_cov, costs, solution)
    history = [float(sol_fitness[1])]
    
    # Initially, first valid index is 0 if starting solution is valid , otherwise it is -1
    first_valid = int(sol_fitness[0] == universe_size) - 1
    for i in tqdm(range(num_steps)):
        curr_cov = best_cov.copy()
        current = tweak(solution.copy(), curr_cov, sets)
        
        curr_fitness = fitness(curr_cov, costs, current)
        history.append(float(curr_fitness[1]))
        
        # Mark current index as first valid (index 0 is the initial solution)
        first_valid = i+1 if curr_fitness[0] == universe_size and first_valid == -1 else first_valid
        
        if curr_fitness > sol_fitness:
            sol_fitness = curr_fitness
            solution = current
            best_cov = curr_cov
            
    ic(first_valid)
    steps_sol = history.index(float(sol_fitness[1]))
    
    ic(sol_fitness)
    ic(steps_sol)
    
    plt.figure(figsize=(14,8))
    plt.plot(
        range(first_valid),
        list(np.full(first_valid, history[first_valid])),
        color="red",
        linestyle="--"
    )
    plt.plot(
        range(first_valid, len(history)),
        list(accumulate(history[first_valid :], max)),
        color="red",
    )
    plt.scatter(range(len(history)), history, marker=".", color="blue")
    
    return solution, sol_fitness, steps_sol
    

### Greedy approach
For each iteration, we collect the set that covers the larger number of still uncovered elements. The iterations continue until complete universe coverage is reached.

In [84]:
def solve_greedy(sets: np.ndarray, costs: np.ndarray, num_sets) -> np.ndarray:
    
    solution = np.full(num_sets, False)
    set_matrix = sets.copy()
    covered = 0

    while covered < set_matrix.shape[1]:
        
        progress = 100 * covered / set_matrix.shape[1]
        print(float(progress), end=" ")
        
        largest_index = np.argmax(set_matrix.sum(axis=1))
        largest = set_matrix[largest_index, :]
        solution[largest_index] = True
        covered += largest.sum()
        
        # For each row of the matrix, set to False the corresponding column if the cell of the "largest" vector is True
        # Given the vector corresponding to the coverings for the larger set (to collect),
        # it removes all possible coverings for those elements covered by this set,
        # in order to ignore them in next steps
        set_matrix *= np.logical_not(largest)
                
    print()
    
    sol_fitness = fitness(np.sum(sets[solution], axis=0), costs, solution)
    
    return solution, sol_fitness

#### Solver caller
Calls solve function for each required instance. 

In [ ]:
NUM_INSTANCES = len(universe_sizes)

# Useful for debugging
MIN_INSTANCE = 1
MAX_INSTANCE = NUM_INSTANCES

for (i, (universe_size, num_sets, density)) in \
    list(enumerate(zip(universe_sizes, num_sets_sizes, densities))) [MIN_INSTANCE-1 : MAX_INSTANCE]:

    ic(f"Generating instance {i+1}")

    sets, costs = generate_data(universe_size, num_sets, density)
    
    num_steps = 100_000 if i > 2 else (10_000 if i == 2 else 1000)

    ic(f"Solving instance {i+1}")
    
    solution, (sol_state, sol_fitness), sol_steps = solve_set_cover(sets, costs, num_steps=num_steps)
    greedy_sol, (greedy_state, greedy_fitness) = solve_greedy(sets, costs, num_sets)
    
    ic(sol_state, sol_fitness)
    ic(greedy_state, greedy_fitness)
    